# Задание 1
В [файле](https://stepik.org/media/attachments/lesson/409319/test1_completed.csv) содержится информация о покупках людей

| id | Товар    | Количество |
|----|----------|------------|
| 1  | Арбуз    | 1.0        |
| 1  | Чай      | 1.0        |
| 1  | Сгущенка | 0.5        |
| 2  | Арбуз    | 3.0        |
| 2  | Чай      | 1.0        |


* id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
* Товар – наименование товара
* Количество – число единиц купленного товара

Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

| 1_Товар | 2_Товар  | Встречаемость |
|---------|----------|---------------|
| Чай     | Арбуз    | 2             |
| Арбуз   | Сгущенка | 1             |
| Чай     | Сгущенка | 1             |


* 1_Товар – наименование первого товара
* 2_Товар – наименование второго товара
* Встречаемость – число раз, когда такая пара была встречена

Другими словами: 2 раза люди покупали одновременно чай и арбуз, 1 раз одновременно покупали арбуз и сгущёнку и 1 раз одновременно были куплены чай со сгущёнкой.

Напишите код на python для получения нужной таблицы и укажите 5 наиболее распространённых паттернов.



### Загрузка данных:

In [1]:
import sys
import pandas as pd
import numpy as np
from itertools import combinations

print('python: ', sys.version)
print('pandas: ', pd.__version__)
print('numpy: ', np.__version__)

python:  3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]
pandas:  1.2.0
numpy:  1.19.5


In [2]:
data = pd.read_csv('test1_completed.csv')
data.columns = ['transaction_id', 'product_name', 'quantity']
data.head()

,transaction_id,product_name,quantity
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


In [3]:
print(data.shape)
print()
data.info()

(43514, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43514 entries, 0 to 43513
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  43514 non-null  int64  
 1   product_name    43514 non-null  object 
 2   quantity        43514 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1020.0+ KB


In [4]:
data.nunique()

transaction_id    3273
product_name       199
quantity           101
dtype: int64

In [5]:
# размеры чеков
data.groupby('transaction_id', as_index=False) \
.agg({'product_name': 'count'}) \
.rename(columns={'product_name': 'count'}) \
.sort_values('count', ascending=False) \
.head(5)

,transaction_id,count
2638,99430,64
670,38766,62
160,23467,61
2123,83977,59
1903,77120,56


In [6]:
# Какие товары чаще всего встречались? 
data.groupby('product_name', as_index=False) \
.agg({'transaction_id': 'count'}) \
.rename(columns={'transaction_id': 'count'}) \
.sort_values('count', ascending=False) \
.head(5)

,product_name,count
113,Огурцы Луховицкие,1022
8,Арбуз,978
165,Укроп,828
15,Бананы,691
55,Кабачки,662


### Решение 1 

Вариант, с помощью которого можно решить задачу на небольших объемах данных.

In [7]:
data.head(5)

,transaction_id,product_name,quantity
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


In [8]:
# формирование комбинаций для каждой транзакции
def get_combinations(transaction):
    products = transaction.sort_values().values
    combs = list(combinations(products, 2))
    return combs

df = data.groupby('transaction_id', as_index=False) \
.agg({'product_name': get_combinations}) \
.rename(columns={'product_name': 'combination'}) \
.explode('combination', ignore_index=True)

df[['product1','product2']] = pd.DataFrame(df['combination'].tolist(), index=df.index)
df.drop(columns=['combination'], inplace=True)
df.head()

,transaction_id,product1,product2
0,17119,Лимон,Лимон оранжевый
1,17119,Лимон,Лук репчатый
2,17119,Лимон,Лук-порей
3,17119,Лимон,Малина свежая
4,17119,Лимон,Морковь немытая


In [9]:
# вычисление встречаемости комбинаций по всем транзакциям
result_baseline = \
df.groupby(['product1', 'product2'], as_index=False) \
.agg({'transaction_id': 'count'}) \
.rename(columns={'transaction_id': 'count'}) \
.sort_values('count', ascending=False)

Результат, 5 наиболее распространенных паттернов:

In [10]:
print(result_baseline.shape)
result_baseline.head(5)

(19697, 3)


,product1,product2,count
16094,Огурцы Луховицкие,Укроп,431
17179,Петрушка,Укроп,408
1660,Арбуз,Огурцы Луховицкие,345
9460,Кабачки,Огурцы Луховицкие,326
11149,Кинза,Укроп,303


### Решение 2: Apriori Algorithm

Попробуем решить эту задачу другим способом, который будет эффективнее на больших датасетах.

[Ссылка на используемые материалы](https://habr.com/ru/company/ods/blog/353502/)

Одна из реализаций алгоритма: 
[ссылка на исходный код библиотеки](https://github.com/ymoch/apyori)

In [11]:
# !pip install apyori
import apyori

print('apyori: ', apyori.__version__)

apyori:  1.1.2


In [12]:
data.head(3)

,transaction_id,product_name,quantity
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0


Подготовка данных для алгоритма:

In [13]:
df = pd.pivot(data=data, index='transaction_id', columns='product_name', values='product_name')
df.head(3)

product_name,Абрикос вяленый,Абрикосы молдавские,Авокадо ХАСС,Авокадо стандарт,Алыча вяленая,Ананас Gold,Ананасовые кольца,Апельсины столовые,Арбуз,Арбуз овальный,...,Яблоки Гала,Яблоки Голден,Яблоки Джонаголд,Яблоки Мутсу,Яблоки Симиренко,Яблоки Фуджи,Яблоки Чемпион,Яблоки сезонные,Яблоки сушеные,Ягоды Годжи
transaction_id,,,,,,,,,,,,,,,,,,,,,
17119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17618,NaN,Абрикосы молдавские,NaN,NaN,NaN,NaN,NaN,Апельсины столовые,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ягоды Годжи


In [14]:
# для самопроверки при формировании таблицы для алгоритма
_df_nunique_before = df.nunique(axis=1)

# замена NA на последнее или первое значение в транзакции
df.fillna(method = 'ffill', inplace = True, axis=1)
df.fillna(method = 'bfill', inplace = True, axis=1)

# для самопроверки при формировании таблицы для алгоритма
_df_nunique_after = df.nunique(axis=1)

assert (_df_nunique_before.values == _df_nunique_after.values).all()

In [15]:
df.head(3)

product_name,Абрикос вяленый,Абрикосы молдавские,Авокадо ХАСС,Авокадо стандарт,Алыча вяленая,Ананас Gold,Ананасовые кольца,Апельсины столовые,Арбуз,Арбуз овальный,...,Яблоки Гала,Яблоки Голден,Яблоки Джонаголд,Яблоки Мутсу,Яблоки Симиренко,Яблоки Фуджи,Яблоки Чемпион,Яблоки сезонные,Яблоки сушеные,Ягоды Годжи
transaction_id,,,,,,,,,,,,,,,,,,,,,
17119,Лимон,Лимон,Лимон,Лимон,Лимон,Лимон,Лимон,Лимон,Лимон,Лимон,...,Черешня сушеная,Черешня сушеная,Черешня сушеная,Черешня сушеная,Черешня сушеная,Черешня сушеная,Черешня сушеная,Черешня сушеная,Черешня сушеная,Черешня сушеная
17530,Бразильский орех,Бразильский орех,Бразильский орех,Бразильский орех,Бразильский орех,Бразильский орех,Бразильский орех,Бразильский орех,Бразильский орех,Бразильский орех,...,Шпинат мини,Шпинат мини,Шпинат мини,Шпинат мини,Шпинат мини,Шпинат мини,Шпинат мини,Шпинат мини,Шпинат мини,Шпинат мини
17618,Абрикосы молдавские,Абрикосы молдавские,Абрикосы молдавские,Абрикосы молдавские,Абрикосы молдавские,Абрикосы молдавские,Абрикосы молдавские,Апельсины столовые,Апельсины столовые,Апельсины столовые,...,Черника свежая,Черника свежая,Черника свежая,Черника свежая,Черника свежая,Черника свежая,Черника свежая,Черника свежая,Черника свежая,Ягоды Годжи


In [16]:
# Executes Apriori algorithm and returns a RelationRecord generator.
# Arguments:
#     transactions -- A transaction iterable object
#                     (eg. [['A', 'B'], ['B', 'C']]).

# Пороговые значения мы выбираем сами в зависимости от того,
# насколько "сильные" правила мы хотим получить

# Keyword arguments:
#     min_support     -- The minimum support of relations (float)      (default = 0.1)
#     min_confidence  -- The minimum confidence of relations (float)   (default = 0.0)
#     min_lift        -- The minimum lift of relations (float)         (default = 0.0)
#     max_length      -- The maximum length of the relation (integer)  (default = None)

results_apriori = list(apyori.apriori(list(df.values), \
                                         max_length = 2, \
                                         min_support = 0.01, \
                                         min_confidence = 0.0, \
                                         min_lift = 0.0))

In [17]:
import json
from io import StringIO


outputs = []
for relation_record in results_apriori:
    output = StringIO()
    apyori.dump_as_json(relation_record, output)
    outputs.append(json.loads(output.getvalue()))
    
result_apriori = pd.DataFrame(outputs)
print(result_apriori.shape)
result_apriori.tail(3)

(2808, 3)


,items,support,ordered_statistics
2805,"[Чеснок молодой, Щавель]",0.017721,"[{'items_base': [], 'items_add': ['Чеснок моло..."
2806,"[Чеснок молодой, Яблоки Симиренко]",0.010999,"[{'items_base': [], 'items_add': ['Чеснок моло..."
2807,"[Шпинат, Щавель]",0.014971,"[{'items_base': [], 'items_add': ['Шпинат', 'Щ..."


Преобразуем результаты:

In [18]:
result_apriori['_items_count'] = result_apriori['items'].apply(lambda items: len(items))
result_apriori.query('_items_count == 2', inplace=True)
result_apriori.reset_index(inplace=True, drop=True)
result_apriori = result_apriori[['items', 'support']]

Результат:

In [19]:
result_apriori.sort_values('support', ascending=False).head(5)

,items,support
2204,"[Огурцы Луховицкие, Укроп]",0.131683
2432,"[Петрушка, Укроп]",0.124656
290,"[Арбуз, Огурцы Луховицкие]",0.105408
1240,"[Кабачки, Огурцы Луховицкие]",0.099603
1514,"[Кинза, Укроп]",0.092576


Результаты аналогичны при решении "в лоб".